Imports:

In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.example_libraries import stax, optimizers
import matplotlib.pyplot as plt
import pyvista as pv
import pinns 
import datetime
import jax.scipy.optimize
import jax.flatten_util
import scipy
import scipy.optimize
import random

from pygccx import model as ccx_model
from pygccx import model_keywords as mk
from pygccx import step_keywords as sk
from pygccx import enums

rnd_key = jax.random.PRNGKey(1234)
np.random.seed(14124)


2024-05-16 17:51:28.475840: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:280] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Set the default precision and the execution device.

In [2]:
jax.config.update("jax_enable_x64", False)
# print("GPU devices: ", jax.devices('gpu'))
dev = jax.devices('gpu')[0] if jax.device_count()>1 and len(jax.devices('gpu'))>0 else jax.devices('cpu')[0]
print(dev)

TFRT_CPU_0


### Geometry definition 

Define the geometry patches:

In [3]:
def get_domain(r0: float, r1: float, R: float, h: float, H: float):

    basis1 = pinns.functions.BSplineBasisJAX(np.array([-1, 0, 1]), 2)
    basis2 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 2)
    basis3 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 1)

    def tmp_gen(angle, r_0, r_1):
        pts = np.zeros([4, 3, 2, 3])
        weights = np.ones([4, 3, 2])

        a = np.pi/2-angle/2
        rs = np.linspace(r_0, r_1, 4)
        pts[-1, 0, 0, :] = [np.cos(-angle/2), np.sin(-angle/2), 0]
        pts[-1, 1, 0, :] = [1/np.sin(a), 0, 0]
        pts[-1, 2, 0, :] = [np.cos(angle/2), np.sin(angle/2), 0]
        pts[0, :, 0, :2] = rs[0] * pts[-1, :, 0, :2]
        pts[1, :, 0, :2] = rs[1] * pts[-1, :, 0, :2]
        pts[2, :, 0, :2] = rs[2] * pts[-1, :, 0, :2]
        pts[3, :, 0, :2] = rs[3] * pts[-1, :, 0, :2]
        pts[0, :, 0, 2] = -1
        pts[1, :, 0, 2] = -1
        pts[2, :, 0, 2] = -1
        pts[3, :, 0, 2] = -1
        pts[:, :, 1, :] = pts[:, :, 0, :]
        pts[:, :, 1, 2] = -pts[:, :, 1, 2]
        weights[:, 1, :] = np.sin(a)

        return pts, weights

    geoms = dict()

    pts, weights = tmp_gen(np.pi/2, r0, r1)
    pts[:, :, :, 2] *= h/2
    # pts[2:,:,:,2] *= h/2
    pts[3, 1, :, 0] = pts[3, 0, :, 0]
    pts[1, 1, :, 0] = 2*pts[0, 1, :, 0]/3+pts[-1, 1, :, 0]/3
    pts[2, 1, :, 0] = pts[0, 1, :, 0]/3+2*pts[-1, 1, :, 0]/3
    weights[-1, 1, :] = 1.0

    geoms['flat'] = pinns.geometry.PatchNURBS(
        [basis1, basis2, basis3], pts.copy(), weights.copy(), 0, 3)

    pts2 = pts[-1, :, :, :]
    weights[...] = 1.0
    linsp = np.linspace(0, 1, basis1.n)

    pts[0, :, :, :] = pts2
    pts[-1, :, :, :] = pts2
    pts[-1, :, :, 0] *= R/r1
    pts[-1, :, :, 1] *= H/h
    # pts[0, :, :, 2] *= H/h

    for i in range(1, basis1.n-1):
        pts[i, :, :, 2] = (1-linsp[i]**0.25)*pts[0, :, :, 2] + \
            linsp[i]**0.25*pts[-1, :, :, 2]
        pts[i, :, :, 0] = (1-linsp[i])*pts[0, :, :, 0] + \
            linsp[i]*pts[-1, :, :, 0]
        pts[i, :, :, 1] = (1-linsp[i]**4)*pts[0, :, :, 1] + \
            linsp[i]**4*pts[-1, :, :, 1]
        pts[i, :, :, 1] *= 2*(linsp[i]-1/2)**2+0.5
        pts[i, :, :, 2] *= 2*(linsp[i]-1/2)**2+0.5

    geoms['spoke'] = pinns.geometry.PatchNURBS(
        [basis1, basis2, basis3], pts, weights, 0, 3)

    pts, weights = tmp_gen(np.pi/2, r0, r1)
    pts[:, :, :, 2] *= h/2
    # pts[2:,:,:,2] *= h/2

    geoms['round_0'] = pinns.geometry.PatchNURBS(
        [basis1, basis2, basis3], pts, weights, 0, 3)
    geoms['round_0'].rotate((0, 0, np.pi/2))

    pts, weights = tmp_gen(np.pi/2, r0, r1)
    pts[:, :, :, 2] *= h/2
    # pts[2:,:,:,2] *= h/2

    geoms['round_1'] = pinns.geometry.PatchNURBS(
        [basis1, basis2, basis3], pts, weights, 0, 3)
    geoms['round_1'].rotate((0, 0, 2*np.pi/2))

    pts, weights = tmp_gen(np.pi/2, r0, r1)
    pts[:, :, :, 2] *= h/2
    # pts[2:,:,:,2] *= h/2

    geoms['round_2'] = pinns.geometry.PatchNURBS(
        [basis1, basis2, basis3], pts, weights, 0, 3)
    geoms['round_2'].rotate((0, 0, 3*np.pi/2))

    return geoms


geoms = get_domain(0.4, 0.8, 3.0, 1.0, 1.5)
names = list(geoms.keys())

Determine the connectivity of the patches:

In [4]:
with jax.disable_jit(True):
    connectivity = pinns.geometry.match_patches(geoms, eps=1e-4, verbose=False)

for c in connectivity:
    print(c)

{'first': 'flat', 'second': 'spoke', 'axis_first': (0,), 'axis_second': (0,), 'end_first': (-1,), 'end_second': (0,), 'axis_permutation': (None, (1, 1), (2, 1))}
{'first': 'flat', 'second': 'round_0', 'axis_first': (1,), 'axis_second': (1,), 'end_first': (-1,), 'end_second': (0,), 'axis_permutation': ((0, 1), None, (2, 1))}
{'first': 'flat', 'second': 'round_2', 'axis_first': (1,), 'axis_second': (1,), 'end_first': (0,), 'end_second': (-1,), 'axis_permutation': ((0, 1), None, (2, 1))}
{'first': 'spoke', 'second': 'round_0', 'axis_first': (0, 1), 'axis_second': (0, 1), 'end_first': (0, -1), 'end_second': (-1, 0), 'axis_permutation': (None, None, (2, 1))}
{'first': 'spoke', 'second': 'round_2', 'axis_first': (0, 1), 'axis_second': (0, 1), 'end_first': (0, 0), 'end_second': (-1, -1), 'axis_permutation': (None, None, (2, 1))}
{'first': 'round_0', 'second': 'round_1', 'axis_first': (1,), 'axis_second': (1,), 'end_first': (-1,), 'end_second': (0,), 'axis_permutation': ((0, 1), None, (2, 1))}

In [9]:
def construct_inp(geoms: dict, E: float, mu: float, dens: float, n: int, meshsize: float, contact_low=None, contact_up=None):
    
    with ccx_model.Model("/usr/bin/ccx", "/usr/bin/cgx", jobname='holder', working_dir="./") as model:

        gmsh = model.get_gmsh()
        
        surfaces = dict()
        
        for name in geoms:
            print(name)
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][:,:,-1](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 2, 0)] = tag 
            
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][:,:,1](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 2, -1)] = tag
            
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][:,-1,:](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 1, 0)] = tag
            
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][:,1,:](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 1, -1)] = tag
            
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][-1,:,:](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 0, 0)] = tag
            
            meshgrid = np.meshgrid(np.linspace(-1,1,n), np.linspace(-1,1,n))
            positions = geoms[name][1,:,:](np.concatenate((meshgrid[0].reshape([-1,1]), meshgrid[1].reshape([-1,1])),-1))
            indices = []
            for i in range(positions.shape[0]):
                tag = gmsh.model.occ.addPoint(positions[i,0], positions[i,1], positions[i,2], tag = -1, meshSize = meshsize)
                indices.append(tag)
            tag = gmsh.model.occ.addBSplineSurface(indices, n, -1, 2, 2)
            surfaces[(name, 0, -1)] = tag
            
       
        #tag = gmsh.model.occ.add_curve_loop([surfaces[('spoke', 0, -1)], surfaces[('spoke', 1, -1)], surfaces[('spoke', 1, 0)], surfaces[('spoke', 2, -1)], surfaces[('spoke', 2, 0)],
        #                                     surfaces[('flat', 0, 0)], surfaces[('flat', 2, 0)], surfaces[('flat', 2, -1)],
        #                                     surfaces[('round_0', 0, -1)], surfaces[('round_0', 0, 0)], surfaces[('round_0', 2, -1)], surfaces[('round_0', 2, 0)],
        #                                     surfaces[('round_1', 0, -1)], surfaces[('round_1', 0, 0)], surfaces[('round_1', 2, -1)], surfaces[('round_1', 2, 0)],
        #                                     surfaces[('round_2', 0, -1)], surfaces[('round_2', 0, 0)], surfaces[('round_2', 2, -1)], surfaces[('round_2', 2, 0)],
        #                                     ])
        
        gmsh.model.occ.remove_all_duplicates()
        closed_surface_loop = [surfaces[('spoke', 0, -1)], surfaces[('spoke', 1, 0)], surfaces[('spoke', 1, -1)], surfaces[('spoke', 2, -1)], surfaces[('spoke', 2, 0)]]
        closed_surface_loop += [surfaces[('flat',0,0)], surfaces[('flat',2,0)], surfaces[('flat',2,-1)]]
        closed_surface_loop += [surfaces[('round_0',0,0)], surfaces[('round_0',0,-1)], surfaces[('round_0',2,0)], surfaces[('round_0',2,-1)]]
        closed_surface_loop += [surfaces[('round_1',0,0)], surfaces[('round_1',0,-1)], surfaces[('round_1',2,0)], surfaces[('round_1',2,-1)]]
        closed_surface_loop += [surfaces[('round_2',0,0)], surfaces[('round_2',0,-1)], surfaces[('round_2',2,0)], surfaces[('round_2',2,-1)]]
        
        tag = gmsh.model.occ.add_surface_loop(closed_surface_loop)
        
        tag_volume = gmsh.model.occ.add_volume([tag])
        
        OFFSET = 1.0
        if contact_low is not None:
            tag_lower_surface = gmsh.model.occ.add_rectangle(-1, -1, 0, 1.5, 2)
            gmsh.model.occ.rotate([(2,tag_lower_surface)], 0, 0, 0, 1, 0, 0, -np.pi/2)
            gmsh.model.occ.translate([(2,tag_lower_surface)], 0, contact_low-OFFSET, 0)
            tags_lower_volume = gmsh.model.occ.extrude([(2, tag_lower_surface)], 0, -0.1, 0)
            tags_lower_volume = [t[1] for t in tags_lower_volume if t[0]==3][0]
            
        if contact_up is not None:
            tag_up_surface = gmsh.model.occ.add_rectangle(-1, -1, 0, 1.5, 2)
            gmsh.model.occ.rotate([(2,tag_up_surface)], 0, 0, 0, 1, 0, 0, -np.pi/2)
            gmsh.model.occ.translate([(2,tag_up_surface)], 0, contact_up+OFFSET, 0)
            tags_up_volume = gmsh.model.occ.extrude([(2, tag_up_surface)], 0, 0.1, 0)
            tags_up_volume = [t[1] for t in tags_up_volume if t[0]==3][0]
            
        gmsh.model.occ.synchronize()
       

        if contact_up is not None: 
            gmsh.model.add_physical_group(2, [surfaces[('round_0', 0, -1)]], name="UPPER_SURFACE")
            gmsh.model.add_physical_group(2, [tag_up_surface], name="UPPER_PRESS_SURFACE")
            gmsh.model.add_physical_group(3, [tags_up_volume], name="UPPER_PRESS")

        if contact_low is not None: 
            gmsh.model.add_physical_group(2, [surfaces[('round_2', 0, -1)]], name="LOWER_SURFACE")
            gmsh.model.add_physical_group(2, [tag_lower_surface], name="LOWER_PRESS_SURFACE")
            gmsh.model.add_physical_group(3, [tags_lower_volume], name="LOWER_PRESS")
            
        gmsh.model.add_physical_group(3, [tag_volume], name='HOLDER')
        gmsh.model.add_physical_group(2, [surfaces[('spoke', 0, -1)]], name='FIX')
        closed_surface_loop.remove(surfaces[('spoke', 0, -1)])
        gmsh.model.add_physical_group(2, closed_surface_loop, name='NOT_FIX')
        
        gmsh.model.mesh.set_size_callback(lambda dim, tag, x, y, z, lc: meshsize)
        gmsh.model.mesh.generate(3)
        
        gmsh.write("test.geo_unrolled")
        gmsh.write("test.msh")
        
        print("meshed")
        
        model.update_mesh_from_gmsh()
        mesh = model.mesh

        
        # fix the left end
        fix_set = model.mesh.get_node_set_by_name('FIX')
        model.add_model_keywords(
            mk.Boundary(fix_set, first_dof=1, last_dof=3)
        )
        if contact_up is not None:
            model.add_model_keywords(mk.Boundary(model.mesh.get_node_set_by_name("UPPER_PRESS_SURFACE"), first_dof=3))
            model.add_model_keywords(mk.Boundary(model.mesh.get_node_set_by_name("UPPER_PRESS_SURFACE"), first_dof=1))
       
        if contact_low is not None:
            model.add_model_keywords(mk.Boundary(model.mesh.get_node_set_by_name("LOWER_PRESS_SURFACE"), first_dof=1))
            model.add_model_keywords(mk.Boundary(model.mesh.get_node_set_by_name("LOWER_PRESS_SURFACE"), first_dof=3))

        mat = mk.Material('ELASTIC')
        el = mk.Elastic((E, mu))
        dens = mk.Density(dens)
        sos = mk.SolidSection(
            elset=mesh.get_el_set_by_name('HOLDER'),
            material = mat
        )
        model.add_model_keywords(mat, el, dens, sos)
        
        if contact_up:
            
            upper_press_surface = mesh.add_surface_from_node_set("UPPER_PRESS_SURFACE_SET", mesh.get_node_set_by_name("UPPER_PRESS_SURFACE"), enums.ESurfTypes.EL_FACE)
            upper_contact_surface = mesh.add_surface_from_node_set("UPPER_SURFACE_SET", mesh.get_node_set_by_name("UPPER_SURFACE"), enums.ESurfTypes.EL_FACE)
            
            mat = mk.Material('ELASTIC')
            el = mk.Elastic((300000., 0.3))
            
            sos = mk.SolidSection(
                elset=mesh.get_el_set_by_name('UPPER_PRESS'),
                material = mat
            )
            model.add_model_keywords(mat, el, sos)
            
            interact = mk.SurfaceInteraction("up")
            behavior = mk.SurfaceBehavior(enums.EPressureOverclosures.LINEAR, k=1e4, sig_inf=1e5)
            pair = mk.ContactPair(interact, enums.EContactTypes.NODE_TO_SURFACE, upper_contact_surface, upper_press_surface)

            model.add_model_keywords(pair, interact, behavior)
            
        if contact_low:
            lower_press_surface = mesh.add_surface_from_node_set("LOWER_PRESS_SURFACE_SET", mesh.get_node_set_by_name("LOWER_PRESS_SURFACE"), enums.ESurfTypes.EL_FACE)
            lower_contact_surface = mesh.add_surface_from_node_set("LOWER_SURFACE_SET", mesh.get_node_set_by_name("LOWER_SURFACE"), enums.ESurfTypes.EL_FACE)
            
            mat = mk.Material('ELASTIC')
            el = mk.Elastic((300000., 0.3))
            
            sos = mk.SolidSection(
                elset=mesh.get_el_set_by_name('LOWER_PRESS'),
                material = mat
            )
            model.add_model_keywords(mat, el, sos)
            
            interact = mk.SurfaceInteraction("low")
            behavior = mk.SurfaceBehavior(enums.EPressureOverclosures.LINEAR, k=1e4, sig_inf=1e5)
            pair = mk.ContactPair(interact, enums.EContactTypes.NODE_TO_SURFACE, lower_contact_surface, lower_press_surface)

            model.add_model_keywords(pair, interact, behavior)
             
        step = sk.Step(nlgeom=True) 
        model.add_steps(step)       
        
        step.add_step_keywords(
            sk.Static(),                # step is a static one
            sk.Dload(mesh.get_el_set_by_name('HOLDER'), enums.EDloadType.GRAV, (9.810, 0, -1, 0)),
            sk.Boundary(model.mesh.get_node_set_by_name("UPPER_PRESS_SURFACE"), 2, -OFFSET),
            sk.Boundary(model.mesh.get_node_set_by_name("LOWER_PRESS_SURFACE"), 2, OFFSET),
            sk.NodeFile([enums.ENodeFileResults.U,
                         enums.ENodeFileResults.RF]), # request deformations in frd file
            sk.ElFile([enums.EElFileResults.S]), # request stresses in frd file
            sk.NodePrint(mesh.get_node_set_by_name('HOLDER'),[enums.ENodePrintResults.U,
                                                            enums.ENodePrintResults.RF]),
            sk.ElPrint(mesh.get_el_set_by_name('HOLDER'),[enums.EElPrintResults.S,
                                                        enums.EElPrintResults.EVOL,
                                                        enums.EElPrintResults.COORD,
                                                        enums.EElPrintResults.E,
                                                        enums.EElPrintResults.ME,
                                                        enums.EElPrintResults.ENER,
                                                        enums.EElPrintResults.ELKE,
                                                        enums.EElPrintResults.ELSE,
                                                        enums.EElPrintResults.EMAS])
        )
        
        model.write_ccx_input_file()
        model.solve(no_cpu=8)
        #model.show_results_in_cgx()
        

        
        # gmsh.write('holder.msh')
        # gmsh.write('holder.inp')
            
    

In [10]:
with jax.disable_jit(True):
    construct_inp(geoms, 2000, 0.3, 0.2, 16, 0.1, -0.75, 0.75)

flat
spoke
round_0
round_1
round_2
Info    : Cannot bind existing OpenCASCADE surface 6 to second tag 11
Info    : Could not preserve tag of 2D object 11 (->6)
Info    : Cannot bind existing OpenCASCADE surface 4 to second tag 15
Info    : Could not preserve tag of 2D object 15 (->4)
Info    : Cannot bind existing OpenCASCADE surface 16 to second tag 21
Info    : Could not preserve tag of 2D object 21 (->16)
Info    : Cannot bind existing OpenCASCADE surface 22 to second tag 27
Info    : Could not preserve tag of 2D object 27 (->22)
Info    : Cannot bind existing OpenCASCADE surface 3 to second tag 28
Info    : Could not preserve tag of 2D object 28 (->3)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (BSpline)
Info    : [ 10%] Meshing curve 2 (BSpline)
Info    : [ 10%] Meshing curve 3 (BSpline)
Info    : [ 10%] Meshing curve 4 (BSpline)
Info    : [ 10%] Meshing curve 5 (BSpline)
Info    : [ 10%] Meshing curve 6 (BSpline)
Info    : [ 10%] Meshing curve 7 (BSpline)
Info    : [

meshed
Info    : Done writing 'test.msh'

************************************************************

CalculiX Version 2.21, Copyright(C) 1998-2023 Guido Dhondt
CalculiX comes with ABSOLUTELY NO WARRANTY. This is free
software, and you are welcome to redistribute it under
certain conditions, see gpl.htm

************************************************************

You are using an executable made on Mi 3. Apr 21:56:04 CEST 2024

  The numbers below are estimated upper bounds

  number of:

   nodes:        13012
   elements:        24016
   one-dimensional elements:            0
   two-dimensional elements:            0
   integration points per element:            1
   degrees of freedom per node:            3
   layers per element:            1

   distributed facial loads:            0
   distributed volumetric loads:            1
   concentrated loads:            0
   single point constraints:         3102
   multiple point constraints:            1
   terms in all multiple poin